In [ ]:
!curl https://ecommwebsitedata.s3.amazonaws.com/acm-sf-chapter-hackathon-big.zip --output bestbuy.zip
!ls
!unzip bestbuy.zip
!tar xvf product_data.tar.gz

In [12]:
import pandas as pd
import xml.etree.ElementTree as ETree
CAT_FILE = "product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml"

In [8]:
cats = pd.read_xml(CAT_FILE)
cats.head()

We need to find a way to read the nested data so that we are able to interpret the actual path data. We can do this using an element tree to store the different levels of nested data.

In [50]:
prstree = ETree.parse(CAT_FILE)
root = prstree.getroot()
all_cats = []

for category in root.iter("category"):
    if len(category) > 2:
        cat_data = []
        name = category.find("name").text
        cat_id = category.find("id").text
        levels = [None for i in range(5)]
        i = 0
        for path in category[2]:
            levels[i] = path.find("id").text
            if i < 4:
                i += 1

        cat_data = [name, cat_id] + levels
        all_cats.append(cat_data)

xmlToDf = pd.DataFrame(all_cats, columns=['name', 'category', 'category_l1', 'category_l2', 'category_l3', 'category_l4', 'category_l5'])
xmlToDf.head()

name      category category_l1   category_l2   category_l3  \
0         Gift Center  abcat0010000    cat00000  abcat0010000          None   
1                 Her  abcat0011000    cat00000  abcat0010000  abcat0011000   
2       Leisure Gifts  abcat0011001    cat00000  abcat0010000  abcat0011000   
3  Kitchen Essentials  abcat0011002    cat00000  abcat0010000  abcat0011000   
4         Electronics  abcat0011003    cat00000  abcat0010000  abcat0011000   

    category_l4 category_l5  
0          None        None  
1          None        None  
2  abcat0011001        None  
3  abcat0011002        None  
4  abcat0011003        None

In [53]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [54]:
updated_train = pd.merge(train, xmlToDf, on='category')
updated_test = pd.merge(test, xmlToDf, on='category')
updated_test.head()
updated_train.head()

user      sku      category  \
0  000000df17cd56a5df4a94074e133c9d4739fae3  2125233  abcat0101001   
1  000001928162247ffaf63185cd8b2a244c78e7c6  2009324  abcat0101001   
2  000017f79c2b5da56721f22f9fdd726b13daf8e8  2877125  abcat0101001   
3  0000c4e9d7075985d1020c456e7ce36f83f834eb  2126065  abcat0101001   
4  00017f7beeac02736c0ce7bf1e75f3010939b34e  1831054  abcat0101001   

                                 query               click_time  \
0  Televisiones Panasonic  50 pulgadas  2011-09-01 23:44:52.533   
1                                Sharp   2011-09-05 12:25:37.42   
2                                  rca  2011-10-25 07:18:14.722   
3                           Samsung 40  2011-09-28 17:27:11.184   
4                               lcd tv  2011-09-28 07:26:32.153   

                query_time                name category_l1   category_l2  \
0  2011-09-01 23:43:59.752  All Flat-Panel TVs    cat00000  abcat0100000   
1  2011-09-05 12:25:01.187  All Flat-Panel TVs    cat00000  abcat0100000   
2  2011-10-25 07:16:51.759  All Flat-Panel TVs    cat00000  abcat0100000   
3  2011-09-28 17:26:15.832  All Flat-Panel TVs    cat00000  abcat0100000   
4  2011-09-28 07:26:21.476  All Flat-Panel TVs    cat00000  abcat0100000   

    category_l3   category_l4 category_l5  
0  abcat0101000  abcat0101001        None  
1  abcat0101000  abcat0101001        None  
2  abcat0101000  abcat0101001        None  
3  abcat0101000  abcat0101001        None  
4  abcat0101000  abcat0101001        None

In [55]:
updated_train.to_csv('new_train.csv')
updated_test.to_csv('new_test.csv')